# Advanced Lane Finding

The purpose of this jupyter notebook is to identify lanelines on a given video stream and save the perspective transformed images along with the fitted polynomial coefficients in a .csv file.


We will start by importing all relevant libraries such as opencv, numpy required.


The Steps:
---

The steps followed in this notebook are the following:

* Apply a distortion correction to raw image frames extracted from video
* Use color transforms, gradients, etc. to create a thresholded binary image
* Apply a perspective transform to the thresholded binary image ("birds-eye view")
* Detect lane pixels and fit a polynomial to the individual pixels to find the laneline
* Save the thresholded perspective transformed image in a separate folder along with the polynomial coefficients in a .csv file

---

## Import necessary libraries

Import all necessary libraries. We will be working with OpenCV, Matplotlib, Glob, Pickle, Natsort and NumPy libraries.

In [1]:
import os
import cv2
import glob
import csv
import natsort
import pickle
import moviepy
import imageio
import progressbar
import time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import moviepy.editor as mpy
from moviepy.editor import VideoFileClip

import multiprocessing as mp

## Thresholding Functions

In this section, I have written functions for calculating parameters such as Sobel Intensity Gradients, Gradient magnitude, gradient direction, hue, lightness, saturation and applying threshold values to idetify lanelines in images. Each of these functions will extract a binary image with the applied threshold values.

The idea behind writing different functions is that there is little extra effort involved and it is possible to use multiple combinations of these filters to achieve the best result.

In [2]:
# Thresholding functions
# since we have evaludated earlier that HLS gives good image filtering results

# this functions accepts a grayscale image as input
def pixel_intensity(img, thresh = (0, 255)):

    # THIS FUNCTION WORKS ONLY ON GRAYSCALE IAMGES!!!
    # 1. apply threshold
    intensity_image = np.zeros_like(img)
    # 2. create a binary image
    intensity_image[(img >= thresh[0]) & (img <= thresh[1])] = 1
    return intensity_image



# this function accepts a HLS format image
def lightness_select(img, thresh = (120,255)):
    
    # 2. Apply threshold to lightness channel
    l_channel = img[:,:,1]
    # 3. Create empty array to store the binary output and apply threshold
    lightness_image = np.zeros_like(l_channel)
    lightness_image[(l_channel >= thresh[0]) & (l_channel <= thresh[1])] = 1
    return lightness_image



# this function accepts a HLS format image
def saturation_select(img, thresh = (100,255)):

    # 2. apply threshold to saturation channel
    s_channel = img[:,:,2]
    # 3. create empty array to store the binary output and apply threshold
    sat_image = np.zeros_like(s_channel)
    sat_image[(s_channel >= thresh[0]) & (s_channel <= thresh[1])] = 1
    return sat_image



# this function accepts a GRAYSCALE format image
# function to create binary image sobel gradients in x and y direction
def abs_sobel_thresh(img, orient = 'x', sobel_kernel = 5, thresh = (0,255)):

    # 1. Applying the Sobel depending on x or y direction and getting the absolute value
    if (orient == 'x'):
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    if (orient == 'y'):
        abs_sobel = np.absolute(cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    # 2. Scaling to 8-bit and converting to np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 3. Create mask of '1's where the sobel magnitude is > thresh_min and < thresh_max
    sobel_image = np.zeros_like(scaled_sobel)
    sobel_image[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sobel_image



# this function accepts a RGB format image
# function to check binary image of sobel magnitude
def mag_sobel(img, sobel_kernel=3, thresh = (0,255)):

    # 1. Applying the Sobel (taking the derivative)
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    # 2. Magnitude of Sobel
    mag_sobel = np.sqrt(sobelx**2 + sobely**2)
    # 3. Scaling to 8-bit and converting to np.uint8
    scaled_sobel = np.uint8(255*mag_sobel/np.max(mag_sobel))
    # 4. Create mask of '1's where the scaled gradient magnitude is > thresh_min and < thresh_max
    sobel_mag_image = np.zeros_like(scaled_sobel)
    sobel_mag_image[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sobel_mag_image



# this function accepts a RGB format image
# function to compute threshold direction
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # 1. Applying the Sobel (taking the derivative)
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    # 2. Take absolute magnitude
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 3. Take Tangent value
    sobel_orient = np.arctan2(abs_sobely, abs_sobelx)
    # 4. Create mask of '1's where the orientation magnitude is > thresh_min and < thresh_max
    dir_image = np.zeros_like(sobel_orient)
    dir_image[(sobel_orient >= thresh[0]) & (sobel_orient <= thresh[1])] = 1
    return dir_image



# this function accepts a HSV format image
# function to compute threshold direction
def value_select(img, thresh=(0,255)):
    
    # 2. apply threshold to saturation channel
    v_channel = img[:,:,2]
    # 3. create empty array to store the binary output and apply threshold
    val_image = np.zeros_like(v_channel)
    val_image[(v_channel > thresh[0]) & (v_channel <= thresh[1])] = 1
    return val_image



# this function uses Canny Edge Detection
# from OpenCV and exports a binary image
def canny_edge(img, thresh=(50, 150)):
    
    canny_image = cv2.Canny(img, thresh[0], thresh[1])
    return canny_image

### Combined Thresholding Function

##### Important:
**Within the combined thresholding function, we call other individual thresholding functions starting from line 13 onwards. The thresholding parameters of all these functions need to be set here.**

We also apply a masking threshold to our image to isolate only the region of interest and remove unneccessary pixel information such as scenery and problematic objects such as vehicle hood. For more details on this, check the [pipeline_mages.ipynb](pipeline_images.ipynb) notebook.

In [3]:
### Combined Thresholding Function
def combined_threshold(img):

    # ----------------------------------- IMAGES PREPROCESSING ------------------------------------- #
    
    # apply Gaussian Blur to remove noise from the image
    img_blurred = cv2.GaussianBlur(img,(3,3),cv2.BORDER_DEFAULT)
    # convert to HLS format
    hls= cv2.cvtColor(img_blurred, cv2.COLOR_RGB2HLS)
    # convert image to grayscale
    gray_blurred = cv2.cvtColor(img_blurred, cv2.COLOR_RGB2GRAY)
    # convert to HSV format
    hsv = cv2.cvtColor(img_blurred, cv2.COLOR_RGB2HSV)
    
    
    '''
    We will calculate the overall pixel intensity of the road by 
    selecting a slice of the road from the center of the image.
    This step will help us to set the correct settings in case 
    of white paved concrete against dark paved concrete paved roads.
    '''

    avg_pixel_intensity = np.mean(gray_blurred[760:810,900:1080])
    avg_pixel_intensity = int(avg_pixel_intensity)

    if (avg_pixel_intensity < 100):
        road_texture = 'dark'
    else:
        road_texture = 'white'
    
    # old values = 155, 130, 180 (pixel intensity, lightness, value )
    # -------------------------- CALL FUNCTIONS FOR THRESHOLDING HERE! ----------------------------- #
    if (road_texture == "dark"):
        pixel_intensity_binary = pixel_intensity(gray_blurred, thresh = (145, 255))
    else:
        pixel_intensity_binary = pixel_intensity(gray_blurred, thresh = (220, 255))
    
    
    if (road_texture == "dark"):
        l_binary = lightness_select(hls, thresh = (120, 255))
    else:
        l_binary = lightness_select(hls, thresh = (160, 255))
    
    
    gradx = abs_sobel_thresh(gray_blurred, orient='x', thresh=(35, 100))
    grady = abs_sobel_thresh(gray_blurred, orient='y', thresh=(35, 100))
    
    # originally 150 and 215 values
    if (road_texture == "dark"):
        v_binary = value_select(hsv, thresh=(150,255))
    else:
        v_binary = value_select(hsv, thresh=(215,255))
        
    
    # ------------------------------ FUNCTION CALLS FOR THRESHOLDING END ------------------------------ #
    #####################################################################################################
    
    # ------------------------------ BEGINNING OF COMBINED THRESHOLDING ------------------------------- #
    
    # combining both x and y gradients
    comb_x_or_y = np.zeros_like(gradx)
    comb_x_or_y[((gradx == 1) | (grady == 1))] = 1
    
    # creating a combined thresholded image
    combined_l_or_intensity = np.zeros_like(pixel_intensity_binary)
    combined_l_or_intensity[((pixel_intensity_binary == 1) | (l_binary == 1))] = 1

    # combined_l_and_intensity = np.zeros_like(pixel_intensity_binary)
    # combined_l_and_intensity[((pixel_intensity_binary == 1) & (l_binary == 1))] = 1

    combined1 = np.zeros_like(gray_blurred)
    combined1[((combined_l_or_intensity == 1) & (v_binary == 1))] = 1
    
    canny_img = canny_edge(gray_blurred, thresh=(50,150))
    
    combined2 = np.zeros_like(gray_blurred)
    combined2[((canny_img == 1) | (comb_x_or_y == 1))] = 1
    
    
    # THIS IS THE FINAL THRESHOLDED IMAGE
    final_thresholded = np.zeros_like(gray_blurred)
    final_thresholded[((combined1 == 1) | (combined2 == 1))] = 1
    
    
    # apply region of interest mask
    height, width = final_thresholded.shape
    mask = np.zeros_like(final_thresholded)
    
    region = np.array([[300, height-112], [880, 710], [1050, 710], [width-200, height-112]], dtype=np.int32)
    cv2.fillPoly(mask, [region], 1)

    masked = cv2.bitwise_and(final_thresholded, mask)

    
    return masked, canny_img

### Perspective Transform

After applying the thresholds, isolating the regions of interest and getting our binary image with identified lanelines, we apply the perspective transform to the image.
We do this using [**cv2.getPerspectiveTransform(src, dst)**](https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html#getperspectivetransform).

This function enables us to obtain a birds-eye view of the lanelines (from top) using which we will calculate lane curvatures.

![image](./readme_images/points_perspective_transform.jpg)

As can be seen in the image above, the following points have been selected to obtain perspective transformed image such that the lanelines appear as parallel.
The pixel locations is selected are (340, 825), (1480, 825), (880, 460) and (1040, 460).

In [4]:
# function for applying perspective view on the images
def perspective_view(img):

    img_size = (img.shape[1], img.shape[0])

    # image points extracted from image approximately
    bottom_left = [480, 960]
    bottom_right = [1445, 960]
    top_left = [900, 710]
    top_right = [1024, 710]

    src = np.float32([bottom_left, bottom_right, top_right, top_left])

    pts = np.array([bottom_left, bottom_right, top_right, top_left], np.int32)
    pts = pts.reshape((-1, 1, 2))

    # choose four points in warped image so that the lines should appear as parallel
    bottom_left_dst = [600, 1080]
    bottom_right_dst = [1300, 1080]
    top_left_dst = [600, 1]
    top_right_dst = [1300, 1]

    dst = np.float32([bottom_left_dst, bottom_right_dst, top_right_dst, top_left_dst])

    # apply perspective transform
    M = cv2.getPerspectiveTransform(src, dst)

    # compute inverse perspective transform
    Minv = cv2.getPerspectiveTransform(dst, src)

    # warp the image using perspective transform M
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    return warped, M, Minv

### Load Saved Pickle Data

Load data from pickle here.

In [5]:
file = open('pickle/dist_pickle.p', 'rb')

# dump information to that file
data = pickle.load(file)

# close the file
file.close()

mtx, dst = data.values()

---

### Lanelines Function

This function contains code for detecting lanelines bases on our approach as explained above. Later on, this function will be used repeatedly for detecting lanelines in our video stream.

I have also included functionality here to store previous lanelines data (2nd order equations) so that they can be used for successive frames. The variables ***avg_left_fit*** and ***avg_right_it*** are calculated based on the measurements of previous 12 frames. This helps the code to fit lanelines for the current frame even if they are not detected.

For a vehicle traveling at maximum speed of 80mph (as per legal limits on most roads in US), the vehicle travels about 35 m/sec. The camera records the video stream at 30 frames/second. Hence, for our weighted averaging of 12 frames/second, the vehicle moves just about 14m (about 3 car lengths) and can be approximated. It can be safely assumed that lanelines will not change significantly during such a smaller distance. This value will be even smaller for lower speeds. Hence, it is safe for approximation.

In [6]:

def advanced_lanelines(img):

    # undistort the original image using stored values from pickle
    undist_original = cv2.undistort(img, mtx, dst, None, mtx)
    
    # apply perspective transform on the thresholded image
    transformed_img, M, Minv = perspective_view(undist_original)
    
    # apply combined threshold
    threshold, canny_img = combined_threshold(img)
    
    # undistort the thresholded image
    undist_thresholded = cv2.undistort(threshold, mtx, dst, None, mtx)
    
    # apply perspective transform on the thresholded image
    warped, M, Minv = perspective_view(undist_thresholded)
    
    # list storing left and right lanefit values from previous frames
    global previous_left_fit
    global previous_right_fit
    
    global prev_left_fits
    global prev_right_fits

    # average of previous 10 lanefits
    global average_left_fit
    global average_right_fit
    
    # previous detection variable
    global previous_detection

    # set this value to True if Radius of the road is to be calculated #
    calculate_radius = False
    
    if (calculate_radius):
        global past_radii

    # initialize the lanelines class by giving inputs from previous iteration
    binary_warped = LaneLines(warped, average_left_fit, average_right_fit, previous_left_fit,\
                              previous_right_fit, previous_detection)

    # calculate the left and right lane fits
    out_img, leftfit, rightfit, detected = binary_warped.find_lane_pixels()

    # we convert our left and right fits from shape (3,) to an array of shape (1,3) to append it to our lists
    previous_left_fit_array = np.array([leftfit])
    previous_right_fit_array = np.array([rightfit])
    previous_detection = detected

    # we add fits from previous detections to our list of previous fits
    prev_left_fits = np.append(prev_left_fits, previous_left_fit_array, axis = 0)
    prev_right_fits = np.append(prev_right_fits, previous_right_fit_array, axis = 0)

    # we ensure that the list doesn't take into account more than 10 previous measurements
    # we delete the initial element of the array if it does, i.e. - earliest element in the array
    if (prev_left_fits.shape[0] > 10):
        prev_left_fits = np.delete(prev_left_fits, 0, axis = 0)
    if(prev_right_fits.shape[0] > 10):
        prev_right_fits = np.delete(prev_right_fits, 0, axis = 0)

    # compute average of past 5 best fits and pass them over to next iteration
    average_left_fit = np.mean(prev_left_fits, axis = 0)
    average_right_fit = np.mean(prev_right_fits, axis = 0)
    
    if (calculate_radius):
        # get the left and right lane radii
        center_offset, left_radius, right_radius = binary_warped.measure_curvature()

        # START OF RADIUS CALCULATIONS
        # calculation of road curvature
        current_road_radius = 0.5*(left_radius+right_radius)

        # Storing past five frame's radii 
        past_radii.append(current_road_radius)
        # calculate weighted average of radii to reduce noisy measurements
        road_radius = sum(past_radii)/len(past_radii)

        # if no outliers are detected then, delete the oldest value
        if (len(past_radii) > 10):
            past_radii.pop(0)

        # calculate mean radius
        road_radius = sum(past_radii)/len(past_radii)
        road_radius = round(road_radius)

        center_offset = round(center_offset, 2)
        road_curvature = "Road Curvature = " + str(road_radius) + "m"
        center_offset = "Center Offset = " + str(center_offset) + "m"
    
    '''
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([binary_warped.left_fitx, binary_warped.ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([binary_warped.right_fitx, binary_warped.ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
    # Draw the lanelines onto the image
    cv2.polylines(color_warp, np.int32([pts_left]), isClosed=False, color=(255,0,0), thickness=20)
    cv2.polylines(color_warp, np.int32([pts_right]), isClosed=False, color=(0,0,255), thickness=20)
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    unwarped = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))

    # Combine the result with the original image
    result = cv2.addWeighted(undist_original, 1, unwarped, 0.3, 0)
    
    '''
    
    previous_left_fit = leftfit
    previous_right_fit = rightfit
    

    # returning original road image, thresholded image, left and right fits
    return transformed_img, warped, leftfit, rightfit

### Run the algorithm on a Video Stream

To run this pipeline on a video stream, we need to import the necessary libraries for video files reading. We will test our pipeline on a short 15 second video file to make sure that all our functions are working correctly and check the output.

The following image is a sample of the generated video that we will get.



In [7]:
# import the lanelines class
from class_lanelines_1 import LaneLines


## Extract images from video files and save polynomial coefficients

Now, we will run this process on all the video files and instead of creating output video files, we will save the individual frame images to a folder. We will run our laneline detection procedure on these image frames and save the polynomial coefficients along with the corresponding image path to a .csv file.

Later on, we will use the saved data to train the neural network model. We will provide the neural network model with the thresholded binary image along with the polynomial coefficients.

We will start by importing the necessary libraries for extracting images.

In [8]:

# directory for video files
file_list = glob.glob("../data_for_lane_detection/videos/*")
total_videos = len(file_list)

# image index number for a million range for large data sets
i = 1000001

print(f"\nImages will be extracted from {total_videos} video files...\n")
print("Video processing started...\n")
print("Images will be extracted and thresholded binary images will be saved ...\n")


# create a directory with the same name and '-images' and 'binary_images' for saving images
try:
    os.mkdir("../data_for_lane_detection/binary_images/")
    print("Binary images will be saved ...")
except FileExistsError:
    print("Folder with the name binary_images already exists!!!")
    print("Warning!!! - Existing files will be overwritten!!!")
    pass

try:
    os.mkdir("../data_for_lane_detection/transformed_images/")
    print("Transformed images will be saved ...")
except FileExistsError:
    print("Folder with the name transformed_images already exists!!!")
    print("Warning!!! - Existing files will be overwritten!!!")
    pass

time.sleep(3.0)

folder_rel_path_transformed = "../data_for_lane_detection/transformed_images/"
folder_rel_path_binary = "../data_for_lane_detection/binary_images/"

####################################################################################################
##################################  Start of Video Processing  #####################################
####################################################################################################

# define functions to save the images
def save_transformed_img(img, img_number, folder_rel_path_transformed):
    transformed_image = cv2.resize(img, None, fx = 0.25, fy = 0.25, interpolation = cv2.INTER_CUBIC)
    transform_file_name = folder_rel_path_transformed + 'transform_img_' + str(img_number) + '.jpg'
    # full absolute path of the file
    trans_abs_path = os.path.abspath(transform_file_name)
    cv2.imwrite(transform_file_name, transformed_image)
    return trans_abs_path

def save_binary_img(img, img_number, folder_rel_path_binary):
    # change resolution by adjusting fx and fy values
    binary_image = cv2.resize(img, None, fx = 0.25, fy = 0.25, interpolation = cv2.INTER_CUBIC)
    # assign file name here ...
    bin_file_name = folder_rel_path_binary + 'bin_img_' + str(i) + '.jpg'
    # full absolute path of the file
    bin_abs_path = os.path.abspath(bin_file_name)
    # path to write binary perspective transformed images to ...
    mpimg.imsave(bin_file_name, binary_image, cmap="gray")
    return bin_abs_path

with open('model_data.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    for item in file_list:
        
        ######## ------- These parameters need to be initialized for every file -------- ########
        start_time = time.time()
        # define variables needed in the global scope
        # these variables store the left and right fits from the previous frame
        previous_left_fit = None
        previous_right_fit = None
        previous_detection = False

        # these variables store the average left and right fits for past 10 frames
        previous_avg_left_fit = None
        previous_avg_right_fit = None

        # initialize empty 1*3 empty arrays for storing lane fit data of previous 10 frames
        prev_left_fits = np.empty([1,3])
        prev_right_fits = np.empty([1,3])

        # intitialize the average left and right fit empty lists - these will be updated in every iteration
        average_left_fit = []
        average_right_fit = []

        # print current video file
        print("Working on video file " + item)
    
        #########################################################################################
        ################ -------- This section deals with the statusbar -------- ################

        # Opens the Video file
        # code for extracting frames starts here ...
        video = cv2.VideoCapture(item)
        video_clip = mpy.VideoFileClip(item)
        frames = int(video_clip.fps * video_clip.duration)

        # this section deals with printing the extraction status
        # code to print status
        frames_per_status_update = int(frames/100)  # update status for evey 2%

        bar = progressbar.ProgressBar(maxval=100, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
        bar.start()

        #########################################################################################
        #########################################################################################
        # no of processed frames
        processed_frames = 0
        # one set of frames processed
        processed_sets = 0
        # number of frames that could not be processed
        error_frame = 0

        while(video.isOpened()):
            # extract frame from the video file
            ret, frame = video.read()
            if(ret == True):

                # run advanced lanelines detection function
                transformed_image, binary_image, left_poly_fit, right_poly_fit = advanced_lanelines(frame)
                
                # We will reduce the resolution by 1/4th in order to reduce overload of GPUs capacity
                
                ###################### SAVE TRANSFORMED IMAGES #######################
                trans_filepath = save_transformed_img(transformed_image, i, folder_rel_path_transformed)
                
                ######################### SAVE BINARY IMAGES #########################
                bin_filepath = save_binary_img(binary_image, i, folder_rel_path_binary)
                
                '''
                ####################### SAVE ORIGINAL IMAGES ########################
                # We will reduce the resolution in order to reduce CNN capacity
                # change resolution by adjusting fx and fy values
                orig_image = cv2.resize(orig_image, None, fx = 0.25, fy = 0.25, interpolation = cv2.INTER_CUBIC)
                # assign file name here ...
                orig_file_name = folder_rel_path_orig + 'orig_img_' + str(i) + '.jpg'
                # full absoulte path of the file
                orig_abs_path = os.path.abspath(orig_file_name)
                # path to write binary perspective transformed images to ...
                mpimg.imsave(orig_file_name, orig_image)
                '''
            else:
                error_frame += 1
                break
                
            spamwriter = csv.writer(csvfile)

            # this line writes the data to the .csv file line by line for every frame
            spamwriter.writerow([trans_filepath] + [bin_filepath] + [left_poly_fit[0]] + [left_poly_fit[1]] + \
                            [left_poly_fit[2]] + [right_poly_fit[0]] + [right_poly_fit[1]] + [right_poly_fit[2]])
            
            # increment image sequence number
            i += 1
            
            # data to be written to the .csv file
            # 1. transformed image
            # 2. binary transformed image
            # 3, 4, 5. - Columns for left laneline
            # 6, 7, 8 - Columns for right laneline

            ##############################################################################
            #################### code for printing extraction statuses ###################
            # increment current frame
            processed_frames = processed_frames + 1

            if(processed_frames == frames_per_status_update):
                processed_sets += 1
                try:
                    bar.update(processed_sets)
                except ValueError:
                    pass
                processed_frames = 0

                
        bar.finish()
            
        print(f"{error_frame-1} frames could not be processed!")

        # release the current video file
        video.release()
        cv2.destroyAllWindows()

        stop_time = time.time()
        
        print(f"Extraction complete for file {item} ...")
        print(f"{frames} images extracted from file {item} ...\n")
        hours, rem = divmod(stop_time-start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print("Time required = " + "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        # print("Time Required = ", "{:.1f}".format(stop_time-start_time), "seconds")
        print("\n")
        
        # end of while loop

# print the summary
num_images = i - 1000001

print(f"{num_images} image data saved for {total_videos} video files ...")
print("Done!!!")


Images will be extracted from 6 video files...

Video processing started...

Images will be extracted and thresholded binary images will be saved ...

Binary images will be saved ...
Transformed images will be saved ...
Working on video file ../data_for_lane_detection/videos\video16.mp4


[========================================================================] 100%


0 frames could not be processed!
Extraction complete for file ../data_for_lane_detection/videos\video16.mp4 ...
409 images extracted from file ../data_for_lane_detection/videos\video16.mp4 ...

Time required = 00:09:22.37


Working on video file ../data_for_lane_detection/videos\video18.mp4


[========================================================================] 100%


0 frames could not be processed!
Extraction complete for file ../data_for_lane_detection/videos\video18.mp4 ...
487 images extracted from file ../data_for_lane_detection/videos\video18.mp4 ...

Time required = 00:05:49.18


Working on video file ../data_for_lane_detection/videos\video4.mp4


...

[========================================================================] 100%


0 frames could not be processed!
Extraction complete for file ../data_for_lane_detection/videos\video5.mp4 ...
1878 images extracted from file ../data_for_lane_detection/videos\video5.mp4 ...

Time required = 00:00:32.02


Working on video file ../data_for_lane_detection/videos\video8.mp4


[========================================================================] 100%


0 frames could not be processed!
Extraction complete for file ../data_for_lane_detection/videos\video8.mp4 ...
1741 images extracted from file ../data_for_lane_detection/videos\video8.mp4 ...

Time required = 00:20:44.12


Working on video file ../data_for_lane_detection/videos\video9.mp4


0 frames could not be processed!
Extraction complete for file ../data_for_lane_detection/videos\video9.mp4 ...
388 images extracted from file ../data_for_lane_detection/videos\video9.mp4 ...

Time required = 00:05:05.24


3101 image data saved for 6 video files ...
Done!!!
